# **Word2Vec with CRNN, CNN, RNN**

Install pyvi for Vietnamese tokenization

In [1]:
!pip install pyvi

## **Import Libraries**

In [3]:
try:
    import tensorflow as tf
except ImportError:
    !pip install tensorflow
    import tensorflow as tf

try:
    import pandas as pd
except ImportError:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except ImportError:
    !pip install numpy
    import numpy as np

try:
    from string import digits
except ImportError:
    !pip install string
    from string import digits

try:
    from collections import Counter
except ImportError:
    !pip install collections
    from collections import Counter

try:
    from pyvi import ViTokenizer
except ImportError:
    !pip install pyvi
    from pyvi import ViTokenizer

try:
    from gensim.models.word2vec import Word2Vec
except ImportError:
    !pip install gensim
    from gensim.models.word2vec import Word2Vec

try:
    from keras.utils import to_categorical
except ImportError:
    !pip install keras
    from keras.utils import to_categorical

# This line is specific to Jupyter Notebooks and does not need a try-except block
%matplotlib inline

In [4]:
from utils.helper_function import *

## **Download Data**

In [5]:
URLs = {
    "https://drive.google.com/file/d/1q3myiaORcL3fbeks8ExZZcqefFtHthPD/view?usp=drive_link": "datasets/vlsp_sentiment_train.csv",
    "https://drive.google.com/file/d/1jofip_UbAXzzJwrqacVTJ7183mmpBQXe/view?usp=drive_link": "datasets/vlsp_sentiment_test.csv",
}

for key, value in URLs.items():
    download_data(key, value)

In [7]:
data_train = pd.read_csv("datasets/vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv("datasets/vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [8]:
print(data_train.shape)
print(data_test.shape)

(5100, 2)
(1050, 2)


In [9]:
labels = data_train.iloc[:, 0].values
reviews = data_train.iloc[:, 1].values

One-hot encode the labels

In [10]:
encoded_labels = []

for label in labels:
    if label == -1:
        encoded_labels.append([1,0,0])
    elif label == 0:
        encoded_labels.append([0,1,0])
    else:
        encoded_labels.append([0,0,1])

encoded_labels = np.array(encoded_labels)  

In [11]:
reviews_processed = []
unlabeled_processed = [] 
for review in reviews:
    review_cool_one = ''.join([char for char in review if char not in digits])
    reviews_processed.append(review_cool_one)

Use PyVi for Vietnamese word tokenizer

In [18]:
word_reviews = []
all_words = []
for review in reviews_processed:
    review = ViTokenizer.tokenize(review.lower())
    word_reviews.append(review.split())
   

In [13]:
EMBEDDING_DIM = 400 # how big is each word vector
MAX_VOCAB_SIZE = 10000 # how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 300 # max number of words in a comment to use

Import tokenizer from keras

In [16]:
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

Tokenize the Vietnamese text

In [19]:
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE, lower=True, char_level=False)
tokenizer.fit_on_texts(word_reviews)
sequences_train = tokenizer.texts_to_sequences(word_reviews)
word_index = tokenizer.word_index


In [20]:
data = pad_sequences(sequences_train, maxlen=MAX_SEQUENCE_LENGTH)
labels = encoded_labels

In [21]:
print('Shape of X train and X validation tensor:',data.shape)
print('Shape of label train and validation tensor:', labels.shape)

Shape of X train and X validation tensor: (5100, 300)
Shape of label train and validation tensor: (5100, 3)


Loading the pre-trained Word2Vec model

In [23]:
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

from gensim.models.keyedvectors import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('checkpoints/vi-model-CBOW.bin', binary=True)

Check the vocabulary size

In [24]:
vocabulary_size=min(len(word_index)+1,MAX_VOCAB_SIZE)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))
print("vocab size", vocabulary_size)

vocab size 7919


Create the embedding matrix

In [25]:
for word, i in word_index.items():
    if i>=MAX_VOCAB_SIZE:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

from keras.layers import Embedding
embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            trainable=True)

Now let's build the CRNN model

In this tutorial, we will use the CRNN model to classify the Vietnamese text. The CRNN model is a combination of CNN and RNN. The CNN is used to extract the features from the text, and the RNN is used to classify the text.

We will build 3 models:

- CNN: Convolutional Neural Network
  + The CNN model consists of 3 convolutional layers with max-pooling layers and dropout layers.
  + The output of the CNN model is fed into a fully connected layer with a softmax activation function.
- RNN: Recurrent Neural Network
- CRNN: Convolutional Recurrent Neural Network

Initialize the layers for the CNN model

In [28]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras.models import Model
from keras import regularizers
sequence_length = data.shape[1]
filter_sizes = [3,4,5]
num_filters = 100
drop = 0.5

In [29]:
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

In [30]:
print(embedding.shape)

(None, 300, 400)


We can see `print(embedding.shape)` is `(None, 300, 400)`. The first dimension is `None` because the batch size is not defined yet. The second dimension is `300` because the maximum length of the text is 300. The third dimension is `400` because the embedding size is 400.

In [ ]:
class CNN:
    def __init__(self):
        self.model 

In [27]:


################## LSTM ONLY ###############################
# reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

################# SINGLE LSTM ####################
# lstm_0 = LSTM(512)(reshape)

# YOU WANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(1024, return_sequences=True)(reshape)
# lstm_1 = LSTM(512, return_sequences=True)(lstm_2)
# lstm_0 = LSTM(256)(lstm_1)

############################################################


################## CRNN ####################################
reshape = Reshape((sequence_length,EMBEDDING_DIM))(embedding)

conv_0 = Conv1D(num_filters, (filter_sizes[0], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_1 = Conv1D(num_filters, (filter_sizes[1], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)
conv_2 = Conv1D(num_filters, (filter_sizes[2], ),padding="same",activation='relu',kernel_regularizer=regularizers.l2(0.01))(reshape)

conv_0 = MaxPool1D(300)(conv_0)
conv_1 = MaxPool1D(300)(conv_1)
conv_2 = MaxPool1D(300)(conv_2)
# Reshape output to match RNN dimension
# conv_0 = Reshape((-1, num_filters))(conv_0)
# conv_1 = Reshape((-1, num_filters))(conv_1)
# conv_2 = Reshape((-1, num_filters))(conv_2)

concat = concatenate([conv_0, conv_1, conv_2])
concat = Flatten()(concat)
# lstm_0 = LSTM(512)(concat)

# YOU WANNA ADD MORE LSTM LAYERS? UNCOMMENT THIS #
# lstm_2 = LSTM(1024, return_sequences=True)(concat)
# lstm_1 = LSTM(512, return_sequences=True)(lstm_2)
# lstm_0 = LSTM(256)(lstm_1)

############################################################

dropout = Dropout(drop)(concat)
output = Dense(units=3, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)

# this creates a model that includes
model = Model(inputs, output)


adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

e:\General_Subjects\Natural Language Processing\Lab-NLP\christ\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


ValueError: Argument(s) not recognized: {'lr': 0.001}

In [ ]:
### IF YOU HAVE MODEL WEIGHT AND WANNA LOAD IT
model.load_weights("model.h5")

In [ ]:
#define callbacks
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.01, patience=4, verbose=1)
callbacks_list = [early_stopping]

model.fit(data, labels, validation_split=0.2,
          epochs=10, batch_size=256, callbacks=callbacks_list, shuffle=True)

Epoch 1/10
16/16 [==============================] - 7s 358ms/step - loss: 7.3026 - accuracy: 0.4544 - val_loss: 6.5601 - val_accuracy: 0.1912
Epoch 2/10
16/16 [==============================] - 5s 322ms/step - loss: 5.3777 - accuracy: 0.6275 - val_loss: 6.6515 - val_accuracy: 0.1000
Epoch 3/10
16/16 [==============================] - 5s 341ms/step - loss: 4.4580 - accuracy: 0.7324 - val_loss: 5.7061 - val_accuracy: 0.1529
Epoch 4/10
16/16 [==============================] - 5s 320ms/step - loss: 3.8115 - accuracy: 0.7882 - val_loss: 5.7683 - val_accuracy: 0.0598
Epoch 5/10
16/16 [==============================] - 5s 320ms/step - loss: 3.2973 - accuracy: 0.8436 - val_loss: 5.2096 - val_accuracy: 0.0824
Epoch 6/10
16/16 [==============================] - 5s 320ms/step - loss: 2.8625 - accuracy: 0.8735 - val_loss: 4.9277 - val_accuracy: 0.0775
Epoch 7/10
16/16 [==============================] - 5s 321ms/step - loss: 2.5052 - accuracy: 0.9020 - val_loss: 4.4247 - val_accuracy: 0.1049
Epoch 

In [ ]:
labels_test = data_test.iloc[:, 0].values
reviews_test = data_test.iloc[:, 1].values

In [ ]:
encoded_labels_test = []

for label_test in labels_test:
    if label_test == -1:
        encoded_labels_test.append([1,0,0])
    elif label_test == 0:
        encoded_labels_test.append([0,1,0])
    else:
        encoded_labels_test.append([0,0,1])

encoded_labels_test = np.array(encoded_labels_test)  

In [ ]:
reviews_processed_test = []
unlabeled_processed_test = [] 
for review_test in reviews_test:
    review_cool_one = ''.join([char for char in review_test if char not in digits])
    reviews_processed_test.append(review_cool_one)

In [ ]:
#Use PyVi for Vietnamese word tokenizer
word_reviews_test = []
all_words = []
for review_test in reviews_processed_test:
    review_test = ViTokenizer.tokenize(review_test.lower())
    word_reviews_test.append(review_test.split())

In [ ]:
sequences_test = tokenizer.texts_to_sequences(word_reviews_test)
data_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
labels_test = encoded_labels_test

In [ ]:
print('Shape of X train and X validation tensor:',data_test.shape)
print('Shape of label train and validation tensor:', labels_test.shape)

Shape of X train and X validation tensor: (1050, 300)
Shape of label train and validation tensor: (1050, 3)


In [ ]:
score = model.evaluate(data_test, labels_test)

33/33 [==============================] - 1s 22ms/step - loss: 2.3534 - accuracy: 0.6133


In [ ]:
print("%s: %.2f" % (model.metrics_names[0], score[0]))
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


loss: 2.35
accuracy: 61.33%


In [ ]:
%cd /content

/content


In [ ]:
model.save_weights("model.h5")

In [ ]:
!cp model.h5 /content/drive/MyDrive